# Synthetic Data Generation for Churn Prediction

This notebook generates synthetic event data tables to support a **churn prediction model** using Databricks declarative feature pipelines.

**Target Schema:** `ananyaroy.feature_store`

**Generated Tables:**
- `user_tenant_mapping` - Maps users to their tenants
- `user_click_events` - Raw click/action events across products
- `confluence_events` - Page views, dwell times, and comments
- `jira_events` - Jira activities and reactions
- `product_entitlements` - Product access records
- `tenant_metrics` - Tenant-level daily aggregated metrics
- `churn_labels` - Binary churn labels for model training

**Use Case:** Predict user churn based on engagement patterns across Atlassian products (Jira, Confluence, JSM, JWM).

## Entity Relationship Diagram

The following diagram shows how the synthetic data tables relate to each other for the churn prediction model:

```mermaid
erDiagram
    USER_TENANT_MAPPING ||--o{ USER_CLICK_EVENTS : "has"
    USER_TENANT_MAPPING ||--o{ CONFLUENCE_EVENTS : "has"
    USER_TENANT_MAPPING ||--o{ JIRA_EVENTS : "has"
    USER_TENANT_MAPPING ||--o{ PRODUCT_ENTITLEMENTS : "has"
    USER_TENANT_MAPPING ||--|| CHURN_LABELS : "has"
    USER_TENANT_MAPPING }o--|| TENANT_METRICS : "belongs to"
    
    USER_TENANT_MAPPING {
        string user_id PK
        string tenant_id FK
    }
    
    USER_CLICK_EVENTS {
        string user_id FK
        string tenant_id FK
        timestamp event_ts
        string action_type
        int duration_seconds
        string product
    }
    
    CONFLUENCE_EVENTS {
        string user_id FK
        string tenant_id FK
        timestamp event_ts
        string page_id
        string event_type
        int dwell_time_seconds
        int comment_count
    }
    
    JIRA_EVENTS {
        string user_id FK
        string tenant_id FK
        timestamp event_ts
        string issue_id
        string activity_type
        int reaction_count
        string jira_product
        string issue_type
    }
    
    PRODUCT_ENTITLEMENTS {
        string user_id FK
        string tenant_id FK
        timestamp entitled_ts
        string product
        boolean is_active
        int page_visits
    }
    
    TENANT_METRICS {
        string tenant_id PK_FK
        timestamp metric_ts
        int key_events_count
        int active_users_count
        int reporting_lines_count
        int linked_aris_count
    }
    
    CHURN_LABELS {
        string user_id PK_FK
        string tenant_id FK
        timestamp label_ts
        int churned
    }
```

**Key Relationships:**
* **user_tenant_mapping** is the central table linking users to tenants
* **Event tables** (user_click_events, confluence_events, jira_events) capture time-series behavioral data
* **product_entitlements** tracks product access and usage
* **tenant_metrics** provides organizational-level context
* **churn_labels** contains the target variable for model training

**Join Keys:**
* `user_id` - Primary key for user-level joins
* `tenant_id` - Foreign key for tenant-level aggregations

## Setup

Initializes configuration parameters and creates the target catalog and schema for storing synthetic data.

**Configuration:**
- **Catalog:** `ananyaroy` - Unity Catalog namespace
- **Schema:** `feature_store` - Schema for all feature tables
- **Users:** 1,000 synthetic users
- **Tenants:** 50 organizations
- **Time Range:** 90 days of historical data

### Initialize Configuration

This cell imports required libraries and sets up configuration parameters:
- Creates the catalog and schema if they don't exist
- Defines the number of synthetic users and tenants
- Sets the time window for generating historical data

In [0]:
import random
from datetime import datetime, timedelta
from pyspark.sql.types import *
from pyspark.sql.functions import col

CATALOG = "ananyaroy"
SCHEMA = "feature_store"
NUM_USERS = 1000
NUM_TENANTS = 50
DAYS_OF_DATA = 90

spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{SCHEMA}")

## Helper Functions

Defines utility functions for generating synthetic data:
- **generate_timestamps()** - Creates random timestamps within a specified date range
- **user_tenant_data** - Generates the foundational user-to-tenant mapping that will be used across all event tables

In [0]:
def generate_timestamps(num_records, days_back):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=days_back)
    return [start_date + timedelta(seconds=random.randint(0, int((end_date - start_date).total_seconds()))) for _ in range(num_records)]

# Generate user-tenant mapping
user_tenant_data = [(f"user_{i}", f"tenant_{random.randint(1, NUM_TENANTS)}") for i in range(1, NUM_USERS + 1)]

## 1. User-Tenant Mapping

**Purpose:** Creates the foundational mapping between users and tenants.

**Schema:**
- `user_id` (String): Unique user identifier
- `tenant_id` (String): Tenant the user belongs to

**Churn Relevance:** This table establishes the user-tenant relationship, which is essential for aggregating tenant-level features and understanding organizational context in churn prediction.

In [0]:
schema = StructType([StructField("user_id", StringType()), StructField("tenant_id", StringType())])
df = spark.createDataFrame(user_tenant_data, schema=schema)
df.write.mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.user_tenant_mapping")
print(f"Created user_tenant_mapping: {df.count()} records")
display(df)

## 2. User Click Events

**Purpose:** Captures raw user interaction events across different products.

**Schema:**
- `user_id` (String): User performing the action
- `tenant_id` (String): Associated tenant
- `event_ts` (Timestamp): When the event occurred
- `action_type` (String): Type of action (click, scroll, submit, navigate)
- `duration_seconds` (Integer): Duration of the action
- `product` (String): Product where action occurred (jira, confluence, jsm, jwm)

**Features Generated:**
- Click count per user
- Action counts by type
- Cross-product visit patterns

**Churn Relevance:** Low engagement (fewer clicks, shorter durations) often signals disengagement and potential churn.

In [0]:
data = []
for user_id, tenant_id in user_tenant_data:
    for ts in generate_timestamps(random.randint(10, 200), DAYS_OF_DATA):
        data.append((user_id, tenant_id, ts, 
                     random.choice(["click", "scroll", "submit", "navigate"]),
                     random.randint(1, 30),
                     random.choice(["jira", "confluence", "jsm", "jwm"])))

schema = StructType([
    StructField("user_id", StringType()),
    StructField("tenant_id", StringType()),
    StructField("event_ts", TimestampType()),
    StructField("action_type", StringType()),
    StructField("duration_seconds", IntegerType()),
    StructField("product", StringType())
])

df = spark.createDataFrame(data, schema=schema)
df.write.mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.user_click_events")
print(f"Created user_click_events: {df.count()} records")
display(df)

## 3. Confluence Events

**Purpose:** Tracks Confluence-specific user activities including page views, dwell time, and collaboration.

**Schema:**
- `user_id` (String): User performing the activity
- `tenant_id` (String): Associated tenant
- `event_ts` (Timestamp): Event timestamp
- `page_id` (String): Confluence page identifier
- `event_type` (String): Type of event (view, dwell, comment, edit, link)
- `dwell_time_seconds` (Integer): Time spent on page
- `comment_count` (Integer): Number of comments made

**Features Generated:**
- User dwell time (confrelevance_user_dwelled)
- Page view counts (confrelevance_user_viewed)
- Comment activity (conf_content_page_comments)

**Churn Relevance:** Declining Confluence engagement (fewer views, less dwell time, no comments) indicates reduced collaboration and higher churn risk.

In [0]:
data = []
for user_id, tenant_id in user_tenant_data:
    for ts in generate_timestamps(random.randint(5, 100), DAYS_OF_DATA):
        data.append((user_id, tenant_id, ts,
                     f"page_{random.randint(1, 500)}",
                     random.choice(["view", "dwell", "comment", "edit", "link"]),
                     random.randint(5, 600),
                     random.randint(0, 10)))

schema = StructType([
    StructField("user_id", StringType()),
    StructField("tenant_id", StringType()),
    StructField("event_ts", TimestampType()),
    StructField("page_id", StringType()),
    StructField("event_type", StringType()),
    StructField("dwell_time_seconds", IntegerType()),
    StructField("comment_count", IntegerType())
])

df = spark.createDataFrame(data, schema=schema)
df.write.mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.confluence_events")
print(f"Created confluence_events: {df.count()} records")
display(df)

## 4. Jira Events

**Purpose:** Captures Jira-specific activities including issue management and reactions.

**Schema:**
- `user_id` (String): User performing the activity
- `tenant_id` (String): Associated tenant
- `event_ts` (Timestamp): Event timestamp
- `issue_id` (String): Jira issue identifier
- `activity_type` (String): Type of activity (create, update, comment, transition, resolve)
- `reaction_count` (Integer): Number of reactions received
- `jira_product` (String): Specific Jira product (jsw, jsm, jwm)
- `issue_type` (String): Type of issue (bug, story, task, epic)

**Features Generated:**
- Product Discovery activity (jira_productDiscovery_activity)
- Maximum reaction counts (max_reaction_count)
- Operational key events (jsw_afj_operational_key_events)

**Churn Relevance:** Reduced Jira activity (fewer issues created/updated, no reactions) signals disengagement from core project management workflows.

In [0]:
data = []
for user_id, tenant_id in user_tenant_data:
    for ts in generate_timestamps(random.randint(5, 150), DAYS_OF_DATA):
        data.append((user_id, tenant_id, ts,
                     f"issue_{random.randint(1, 1000)}",
                     random.choice(["create", "update", "comment", "transition", "resolve"]),
                     random.randint(0, 20),
                     random.choice(["jsw", "jsm", "jwm"]),
                     random.choice(["bug", "story", "task", "epic"])))

schema = StructType([
    StructField("user_id", StringType()),
    StructField("tenant_id", StringType()),
    StructField("event_ts", TimestampType()),
    StructField("issue_id", StringType()),
    StructField("activity_type", StringType()),
    StructField("reaction_count", IntegerType()),
    StructField("jira_product", StringType()),
    StructField("issue_type", StringType())
])

df = spark.createDataFrame(data, schema=schema)
df.write.mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.jira_events")
print(f"Created jira_events: {df.count()} records")
display(df)

## 5. Product Entitlements

**Purpose:** Tracks which products users have access to and their usage patterns.

**Schema:**
- `user_id` (String): User with entitlement
- `tenant_id` (String): Associated tenant
- `entitled_ts` (Timestamp): When entitlement was granted
- `product` (String): Product name (jira, confluence, jsm, jwm, cac_page, product_discovery)
- `is_active` (Boolean): Whether entitlement is currently active
- `page_visits` (Integer): Number of visits to product pages

**Features Generated:**
- CAC Page entitlement (Product_entitlement_CACPage)
- Setup experience entitlement (jira_entitlement_setupExperience)

**Churn Relevance:** Inactive entitlements or low page visits despite having access indicates lack of product adoption, a strong churn predictor.

In [0]:
products = ["jira", "confluence", "jsm", "jwm", "cac_page", "product_discovery"]
data = []
for user_id, tenant_id in user_tenant_data:
    for product in random.sample(products, random.randint(1, len(products))):
        ts = datetime.now() - timedelta(days=random.randint(30, DAYS_OF_DATA))
        data.append((user_id, tenant_id, ts, product,
                     random.choice([True, True, True, False]),
                     random.randint(0, 100)))

schema = StructType([
    StructField("user_id", StringType()),
    StructField("tenant_id", StringType()),
    StructField("entitled_ts", TimestampType()),
    StructField("product", StringType()),
    StructField("is_active", BooleanType()),
    StructField("page_visits", IntegerType())
])

df = spark.createDataFrame(data, schema=schema)
df.write.mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.product_entitlements")
print(f"Created product_entitlements: {df.count()} records")
display(df)

## 6. Tenant Metrics

**Purpose:** Aggregates daily tenant-level metrics to capture organizational health signals.

**Schema:**
- `tenant_id` (String): Tenant identifier
- `metric_ts` (Timestamp): Date of metrics
- `key_events_count` (Integer): Count of important events
- `active_users_count` (Integer): Number of active users
- `reporting_lines_count` (Integer): Organizational structure metric
- `linked_aris_count` (Integer): Count of linked artifacts

**Features Generated:**
- Tenant-level operational key events (jsw_afj_operational_key_events_tenant_counts)
- Reporting lines (cg_reporting_lines)
- Linked artifacts (confluence_linked_aris)

**Churn Relevance:** Declining tenant-level metrics (fewer active users, reduced key events) indicate organizational disengagement and higher churn probability.

In [0]:
data = []
for tenant_num in range(1, NUM_TENANTS + 1):
    tenant_id = f"tenant_{tenant_num}"
    base_date = datetime.now() - timedelta(days=DAYS_OF_DATA)
    for day in range(DAYS_OF_DATA):
        ts = base_date + timedelta(days=day)
        data.append((tenant_id, ts,
                     random.randint(10, 500),
                     random.randint(5, 200),
                     random.randint(0, 50),
                     random.randint(100, 5000)))

schema = StructType([
    StructField("tenant_id", StringType()),
    StructField("metric_ts", TimestampType()),
    StructField("key_events_count", IntegerType()),
    StructField("active_users_count", IntegerType()),
    StructField("reporting_lines_count", IntegerType()),
    StructField("linked_aris_count", IntegerType())
])

df = spark.createDataFrame(data, schema=schema)
df.write.mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.tenant_metrics")
print(f"Created tenant_metrics: {df.count()} records")
display(df)

## 7. Churn Labels

**Purpose:** Generates binary churn labels for supervised learning.

**Schema:**
- `user_id` (String): User identifier
- `tenant_id` (String): Associated tenant
- `label_ts` (Timestamp): Observation date for the label
- `churned` (Integer): Binary label (1 = churned, 0 = retained)

**Churn Relevance:** This is the target variable for the churn prediction model. Approximately 15% of users are labeled as churned to simulate realistic class imbalance.

In [0]:
observation_date = datetime.now()
data = [(user_id, tenant_id, observation_date, 1 if random.random() < 0.15 else 0) 
        for user_id, tenant_id in user_tenant_data]

schema = StructType([
    StructField("user_id", StringType()),
    StructField("tenant_id", StringType()),
    StructField("label_ts", TimestampType()),
    StructField("churned", IntegerType())
])

df = spark.createDataFrame(data, schema=schema)
df.write.mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.churn_labels")
print(f"Created churn_labels: {df.count()} records")
print(f"Churn rate: {df.filter(col('churned') == 1).count() / df.count():.2%}")
display(df)

## Verify Tables

Lists all created tables in the target schema to confirm successful data generation.

All tables should be present in the `ananyaroy.feature_store` schema and ready for use in the feature engineering pipeline.

In [0]:
display(spark.sql(f"SHOW TABLES IN {CATALOG}.{SCHEMA}"))